In [1]:
import pandas as pd; import numpy as np
from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import * 

In [2]:
spark = SparkSession.builder.\
        appName('spark_external_data').\
        getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/28 20:31:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('/Users/xwyang/Desktop/data/flight_delays.csv',header=True,inferSchema=True)

In [4]:
df.columns

['date', 'delay', 'distance', 'origin', 'destination']

In [5]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [6]:
spark.sql('create database if not exists my_spark_db')

DataFrame[]

In [7]:
spark.sql('use my_spark_db')

DataFrame[]

In [8]:
spark.sql('show tables').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [10]:
df.write.saveAsTable('flys_tbl_1')

In [11]:
spark.sql('show tables').show()

+-----------+----------+-----------+
|  namespace| tableName|isTemporary|
+-----------+----------+-----------+
|my_spark_db|flys_tbl_1|      false|
+-----------+----------+-----------+



In [12]:
df.createOrReplaceTempView('fly_delays_tbl')

In [13]:
spark.sql('show tables').show()

+-----------+--------------+-----------+
|  namespace|     tableName|isTemporary|
+-----------+--------------+-----------+
|my_spark_db|    flys_tbl_1|      false|
|           |fly_delays_tbl|       true|
+-----------+--------------+-----------+



In [14]:
spark.sql(""" create table if not exists flight_delay_tbl(date string,
 delay int, distance int, origin string, destination string)
            using csv options(path '/Users/xwyang/Desktop/data/flight_delays.csv',
            header=True, inferSchema=True)""")

DataFrame[]

In [15]:
spark.sql('select * from flight_delay_tbl limit 3').show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+



In [16]:
df.count()

1391578

In [17]:
spark.sql("show tables").show()

+-----------+----------------+-----------+
|  namespace|       tableName|isTemporary|
+-----------+----------------+-----------+
|my_spark_db|flight_delay_tbl|      false|
|my_spark_db|      flys_tbl_1|      false|
|           |  fly_delays_tbl|       true|
+-----------+----------------+-----------+



In [18]:
spark.sql('alter table flys_tbl_1 add column status_update string')

DataFrame[]

In [19]:
spark.sql("select * from flys_tbl_1 limit 2").show()

+-------+-----+--------+------+-----------+-------------+
|   date|delay|distance|origin|destination|status_update|
+-------+-----+--------+------+-----------+-------------+
|1010630|  -10|     928|   RSW|        EWR|         NULL|
|1021029|   87|     974|   RSW|        ORD|         NULL|
+-------+-----+--------+------+-----------+-------------+



## To connect to a MySQL database via spark SQL

 ### spark = SparkSession\
          .builder\
          .appName('mysql_spark')\
          .config("spark.jars","/path/to/mysql-connector-java-8.0.26.jar")\
          .getOrCreate()
 . Load  data from and save it to MySQL database using the Spark SQL  data source API and JDBC 

  ### spark.read.format('jdbc')\
       .option('url','jdbc:mysql://[hostname]:3306/[mydatabase]')\
       .option('driver','com.mysql.cj.jdbc.Driver')\
       .option('dbtable','[tablename]')\
       .option('user','[username]')\
       .option('password','[password]')\
       .load()
 
 connection_properties={
 
### spark.read.jdbc
        (url=mysql_url, 
         table='mytable',
         properties=connection_properties, 
         mode='append',# 'overwrite,error,ignor'
         numPartition=10,
         lowerBound=0,
         upperBound=10000,
         parttionColumn='id')
         
 
from pyspark.sql import SparkSession

## Initialize SparkSession with the MySQL JDBC driver specified in the spark.jars configuration
spark = SparkSession.builder \
    .appName("Write_DataFrame_to_JDBC") \
    .config("spark.jars", "/path/to/mysql-connector-java-8.0.27.jar") \
    .getOrCreate()

## MySQL connection properties
mysql_url = "jdbc:mysql://localhost:3306/mydatabase"

mysql_properties = {
    "user": "root",
    "password": "password",
    "driver": "com.mysql.cj.jdbc.Driver"
}

## Define partitioning properties
partitionColumn = "id"  # The column to partition by

numPartitions = 10

lowerBound = 1

upperBound = 1000000

## Read data from MySQL table into a Spark DataFrame with partitioning
df = spark.read.jdbc(

    url=mysql_url,
    table="employees",
    properties=mysql_properties,
    column=partitionColumn,
    lowerBound=lowerBound,
    upperBound=upperBound,
    numPartitions=numPartitions
)

## Repartition the DataFrame
df = df.repartition(numPartitions)

## Define JDBC write properties
jdbc_write_properties = {
    "user": "root",
    "password": "password",
    "driver": "com.mysql.cj.jdbc.Driver"
}

## Define the URL for the JDBC connection
jdbc_url = "jdbc:mysql://localhost:3306/mydatabase"

## Write the DataFrame to the MySQL database table with partitioning
df.write.jdbc(
    url=jdbc_url,
    table="employees",
    mode="overwrite",
    properties=jdbc_write_properties
)


In [23]:

# Sample data
data = [
    Row(department="HR", employee_name="Alice"),
    Row(department="HR", employee_name="Bob"),
    Row(department="IT", employee_name="Charlie"),
    Row(department="IT", employee_name="David"),
    Row(department="IT", employee_name="Eve")
]

# Create DataFrame
df = spark.createDataFrame(data)
df.createOrReplaceTempView("employees")

# Show the DataFrame
df.show()


+----------+-------------+
|department|employee_name|
+----------+-------------+
|        HR|        Alice|
|        HR|          Bob|
|        IT|      Charlie|
|        IT|        David|
|        IT|          Eve|
+----------+-------------+



In [26]:
new_df = spark.sql(""" select department,collect_list(employee_name) as employee_names 
          from employees group by department """)

In [29]:
new_df.show(truncate=False)

+----------+---------------------+
|department|employee_names       |
+----------+---------------------+
|HR        |[Alice, Bob]         |
|IT        |[Charlie, David, Eve]|
+----------+---------------------+



In [31]:
new_df.select('department',explode(col('employee_names')).alias('employee_name')).show()

+----------+-------------+
|department|employee_name|
+----------+-------------+
|        HR|        Alice|
|        HR|          Bob|
|        IT|      Charlie|
|        IT|        David|
|        IT|          Eve|
+----------+-------------+



In [32]:
@pandas_udf('long')
def plus_one(v):
    return v+1


In [33]:
dw = spark.range(5)
dw.withColumn('plus_one',plus_one('id')).show()

+---+--------+
| id|plus_one|
+---+--------+
|  0|       1|
|  1|       2|
|  2|       3|
|  3|       4|
|  4|       5|
+---+--------+



/Users/xwyang/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
/Users/xwyang/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
/Users/xwyang/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
/Users/xwyang/anaconda3/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. 

In [34]:
df.columns

['department', 'employee_name']

In [35]:
new_df.columns

['department', 'employee_names']

In [37]:
da = spark.sql("select * from flys_tbl_1")

In [38]:
da.show(2)

+-------+-----+--------+------+-----------+-------------+
|   date|delay|distance|origin|destination|status_update|
+-------+-----+--------+------+-----------+-------------+
|1010630|  -10|     928|   RSW|        EWR|         NULL|
|1021029|   87|     974|   RSW|        ORD|         NULL|
+-------+-----+--------+------+-----------+-------------+
only showing top 2 rows



In [40]:
dd = da.groupBy('origin').agg(collect_list(col('delay')).alias('delay_orgi')).orderBy('delay_orgi')

In [43]:
dd.show(2)

+------+--------------------+
|origin|          delay_orgi|
+------+--------------------+
|   YAK|[-25, -42, -14, 0...|
|   OME|[-21, -18, -13, -...|
+------+--------------------+
only showing top 2 rows



In [44]:
dd.select('origin',explode('delay_orgi')).show(2)

24/05/28 22:41:21 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB


+------+---+
|origin|col|
+------+---+
|   YAK|-25|
|   YAK|-42|
+------+---+
only showing top 2 rows



In [45]:
dd.show(2)

+------+--------------------+
|origin|          delay_orgi|
+------+--------------------+
|   YAK|[-25, -42, -14, 0...|
|   OME|[-21, -18, -13, -...|
+------+--------------------+
only showing top 2 rows



In [47]:
dd.select('origin',array_min('delay_orgi')).show(2)

+------+---------------------+
|origin|array_min(delay_orgi)|
+------+---------------------+
|   YAK|                  -53|
|   OME|                  -28|
+------+---------------------+
only showing top 2 rows



In [48]:
dd.printSchema()

root
 |-- origin: string (nullable = true)
 |-- delay_orgi: array (nullable = false)
 |    |-- element: integer (containsNull = false)



In [49]:
spark.sql( " select array_distinct(array(1,2,3,null,3))").show()

+---------------------------------------+
|array_distinct(array(1, 2, 3, NULL, 3))|
+---------------------------------------+
|                        [1, 2, 3, NULL]|
+---------------------------------------+



In [56]:
spark.sql("select sequence(to_date('2022-01-01'),to_date('2022-01-5'),interval 1 day) as date ").show(2,False)

+------------------------------------------------------------+
|date                                                        |
+------------------------------------------------------------+
|[2022-01-01, 2022-01-02, 2022-01-03, 2022-01-04, 2022-01-05]|
+------------------------------------------------------------+

